In [1]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import pathlib
import random
import string
import re
import numpy as np
import tensorflow as tf
import tensorflow.data as tf_data
import tensorflow.strings as tf_strings

import keras
from keras import layers
#from keras import ops
from keras.layers import TextVectorization
from keras import backend as K

from tqdm.auto import tqdm

In [2]:
barnette_graphs=np.load('barnette_graph_data_alpha.npy',allow_pickle=True)

In [3]:
barnette_graphs[2]

'13.14.15.16 5.6.2.1 2.6.8.4.10.9 3.4.8.7 5.1.12.11.3.7 5.6.8.7 10.4.3.11.14.13 12.1.2.9.16.15 14.11.12.15 16.9.10.13'

In [4]:
barnette_graphs_dicts=[]
for i in range(len(barnette_graphs)):
    face_info_dot_seperated=barnette_graphs[i].split(' ')
    face_vertex_dict={}
    for k in range(len(face_info_dot_seperated)):
        num_list=[eval(i) for i in face_info_dot_seperated[k].split('.')]
        face_vertex_dict[str(k+1)]=num_list
    barnette_graphs_dicts.append(face_vertex_dict)

In [5]:
hamiltonian_cycles=np.load('hamiltonian_data_alpha.npy',allow_pickle=True)

In [6]:
hamiltonian_cycles[2]

array([1., 0., 1., 0., 1., 0., 1., 0., 0., 0.])

In [7]:
len(barnette_graphs)

50000

In [8]:
len(hamiltonian_cycles)

50000

In [9]:
max_barnette_len=0
for j in range(len(barnette_graphs)):
    if len(barnette_graphs[j])>max_barnette_len:
        max_barnette_len=len(barnette_graphs[j])

In [10]:
max_barnette_len

368

In [11]:
barnette_vocab=[
'0',
'1',
'2',
'3',
'4',
'5',
'6',
'7',
'8',
'9',
' ',
'.',
'p',
''
]

In [12]:
barnette_vocab_dict={}
for i in range(len(barnette_vocab)):
    barnette_vocab_dict[barnette_vocab[i]]=i

In [13]:
barnette_vocab_rev_dict={}
for i in range(len(barnette_vocab)):
    barnette_vocab_rev_dict[i]=barnette_vocab[i]

In [14]:
barnette_vocab_dict

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9,
 ' ': 10,
 '.': 11,
 'p': 12,
 '': 13}

In [15]:
barnette_vocab_rev_dict

{0: '0',
 1: '1',
 2: '2',
 3: '3',
 4: '4',
 5: '5',
 6: '6',
 7: '7',
 8: '8',
 9: '9',
 10: ' ',
 11: '.',
 12: 'p',
 13: ''}

In [16]:
def text_to_numeric(data,vocab_dict,max_len):
    data_numeric=[]
    for i in range(len(data)):
        numeric=[]

        for char in data[i]:
            numeric.append(vocab_dict[char])

        count=len(numeric)

        while count<=max_len:
            numeric.append(vocab_dict['p'])
            count=count+1
        data_numeric.append(np.array(numeric))

    return np.array(data_numeric)

In [17]:
barnette_graphs_numeric=text_to_numeric(barnette_graphs,barnette_vocab_dict, 400)

In [18]:
barnette_graphs_numeric.shape

(50000, 401)

In [19]:
barnette_graphs_numeric[2]

array([ 1,  3, 11,  1,  4, 11,  1,  5, 11,  1,  6, 10,  5, 11,  6, 11,  2,
       11,  1, 10,  2, 11,  6, 11,  8, 11,  4, 11,  1,  0, 11,  9, 10,  3,
       11,  4, 11,  8, 11,  7, 10,  5, 11,  1, 11,  1,  2, 11,  1,  1, 11,
        3, 11,  7, 10,  5, 11,  6, 11,  8, 11,  7, 10,  1,  0, 11,  4, 11,
        3, 11,  1,  1, 11,  1,  4, 11,  1,  3, 10,  1,  2, 11,  1, 11,  2,
       11,  9, 11,  1,  6, 11,  1,  5, 10,  1,  4, 11,  1,  1, 11,  1,  2,
       11,  1,  5, 10,  1,  6, 11,  9, 11,  1,  0, 11,  1,  3, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12

In [20]:
barnette_graphs_numeric=tf.cast(barnette_graphs_numeric,tf.int64)

In [21]:
max_hamiltonian_len=0
for i in range(len(hamiltonian_cycles)):
    if len(hamiltonian_cycles[i])>max_hamiltonian_len:
        max_hamiltonian_len=len(hamiltonian_cycles[i])

In [22]:
#hamiltonian_cycles_padded=np.zeros((len(hamiltonian_cycles),max_hamiltonian_len))-1
hamiltonian_cycles_padded=np.zeros((len(hamiltonian_cycles),max_hamiltonian_len))
for i in range(len(hamiltonian_cycles)):
    for j in range(len(hamiltonian_cycles[i])):
        hamiltonian_cycles_padded[i][j]=hamiltonian_cycles[i][j]

In [23]:
hamiltonian_cycles_padded[2]

array([1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])

In [24]:
max_face_number=0
for i in range(len(barnette_graphs_dicts)):
    if len(barnette_graphs_dicts[i])>max_face_number:
        max_face_number=len(barnette_graphs_dicts[i])

In [25]:
max_face_number

24

In [26]:
max_face_size=0
for i in range(len(barnette_graphs_dicts)):
    for key in barnette_graphs_dicts[i].keys():
        if len(barnette_graphs_dicts[i][key])>max_face_size:
            max_face_size=len(barnette_graphs_dicts[i][key])

In [27]:
max_face_size

22

In [28]:
barnette_graphs_padded=np.zeros((len(barnette_graphs),max_face_number,max_face_size))
for i in range(len(barnette_graphs_dicts)):
    for key in barnette_graphs_dicts[i].keys():
        face_index=int(key)-1
        face=barnette_graphs_dicts[i][key]
        for j in range(len(face)):
            barnette_graphs_padded[i][face_index][j]=barnette_graphs_dicts[i][key][j]

In [29]:
barnette_graphs_padded=tf.cast(barnette_graphs_padded,tf.int64)

In [30]:
barnette_graphs_padded.shape

TensorShape([50000, 24, 22])

In [31]:
pseudolabel=[]
for i in range(len(barnette_graphs_padded)):
    pseudolabel.append(np.array([1,0]))
pseudolabel=tf.cast(pseudolabel,tf.float32)

In [32]:
pseudolabel.shape

TensorShape([50000, 2])

In [33]:
num_val_samples = int(0.15 * len(hamiltonian_cycles))
num_train_samples = len(hamiltonian_cycles) - 2 * num_val_samples

barnette_graphs_trn = barnette_graphs_numeric[:num_train_samples]
hamiltonian_cycles_trn = hamiltonian_cycles_padded[:num_train_samples]
barnette_graphs_padded_trn = barnette_graphs_padded[:num_train_samples]
pseudolabel_trn=pseudolabel[:num_train_samples]

barnette_graphs_val = barnette_graphs_numeric[num_train_samples : num_train_samples + num_val_samples]
hamiltonian_cycles_val = hamiltonian_cycles_padded[num_train_samples : num_train_samples + num_val_samples]
barnette_graphs_padded_val = barnette_graphs_padded[num_train_samples : num_train_samples + num_val_samples]
pseudolabel_val=pseudolabel[num_train_samples : num_train_samples + num_val_samples]


barnette_graphs_tst = barnette_graphs_numeric[num_train_samples + num_val_samples :]
hamiltonian_cycles_tst = hamiltonian_cycles_padded[num_train_samples + num_val_samples :]
barnette_graphs_padded_tst = barnette_graphs_padded[num_train_samples + num_val_samples :]
pseudolabel_tst=pseudolabel[num_train_samples + num_val_samples :]



print(f"{len(hamiltonian_cycles)} total samples")
print(f"{len(hamiltonian_cycles_trn)} training samples")
print(f"{len(hamiltonian_cycles_val)} validation pairs")
print(f"{len(hamiltonian_cycles_tst)} test pairs")

50000 total samples
35000 training samples
7500 validation pairs
7500 test pairs


In [34]:
#import keras.ops as ops

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:,tf.newaxis, tf.newaxis, :], dtype='int32')
        else:
            padding_mask = None

        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "dense_dim": self.dense_dim,
                "num_heads": self.num_heads,
            }
        )
        return config


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        if mask is None:
            return None
        else:
            return tf.math.not_equal(inputs,0)
        #ops.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "sequence_length": self.sequence_length,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(latent_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, None, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = None

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, None]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.convert_to_tensor([1, 1])],
            axis=0,
        )
        return tf.tile(mask, mult)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "latent_dim": self.latent_dim,
                "num_heads": self.num_heads,
            }
        )
        return config


In [35]:
def planar_hamiltonian_loss_(y_true, y_pred):
    m=y_pred
    m=m>0.5
    m=tf.cast(m,tf.int64)
    
    n=barnette_graph_padded_input
    n=tf.cast(n, dtype=tf.dtypes.int64)
    
    I=tf.eye(hamiltonian_cycles_padded[0].shape[0], dtype=tf.dtypes.int64)
    J=tf.map_fn(lambda m : m*I, elems=(m))
    K=tf.matmul(J,n)
    
    K=tf.map_fn(lambda i: tf.reshape(i,[-1]), elems=(K))
    
    vertex_count= tf.map_fn(lambda i : tf.cast([tf.reduce_max(i)],dtype=tf.int64) , elems=(n))
        
    boundary_vertices=tf.map_fn(lambda i : tf.cast([tf.math.reduce_sum(tf.cast(tf.unique_with_counts(i)[2]<=2,tf.int64))],tf.int64) , elems=(K))
        
    interior_vertices=tf.map_fn(lambda i : tf.cast([tf.math.reduce_sum(tf.cast(tf.unique_with_counts(i)[2]>2,tf.int64))-1],dtype=tf.int64) , elems=(K))
    
    boundary_vertices_proportion= boundary_vertices/vertex_count
    interior_vertices_proportion= interior_vertices/vertex_count
        
    y_pred=tf.concat([boundary_vertices_proportion,interior_vertices_proportion],axis=1)
    
    l=-(1-y_pred[0][:,0])*tf.math.log((1-y_pred[0][:,0]))-(y_pred[0][:,1])*tf.math.log((y_pred[0][:,1]))
    

        
    return l

In [36]:
def y_true_calculator(y_true_logits,barnette_graph_padded_input):
    m=y_true_logits
    m=m>0.5
    m=tf.cast(m,tf.int64)
    
    n=barnette_graph_padded_input
    n=tf.cast(n, dtype=tf.dtypes.int64)
    
    I=tf.eye(hamiltonian_cycles_padded[0].shape[0], dtype=tf.dtypes.int64)
    J=tf.map_fn(lambda m : m*I, elems=(m))
    K=tf.matmul(J,n)
    
    K=tf.map_fn(lambda i: tf.reshape(i,[-1]), elems=(K))
    
    vertex_count= tf.map_fn(lambda i : tf.cast([tf.reduce_max(i)],dtype=tf.int64) , elems=(n))
        
    boundary_vertices=tf.map_fn(lambda i : tf.cast([tf.math.reduce_sum(tf.cast(tf.unique_with_counts(i)[2]<=2,tf.int64))],tf.int64) , elems=(K))
        
    interior_vertices=tf.map_fn(lambda i : tf.cast([tf.math.reduce_sum(tf.cast(tf.unique_with_counts(i)[2]>2,tf.int64))-1],dtype=tf.int64) , elems=(K))
    
    boundary_vertices_proportion= boundary_vertices/vertex_count
    interior_vertices_proportion= interior_vertices/vertex_count
        
    y_true=tf.concat([boundary_vertices_proportion,interior_vertices_proportion],axis=1)
    
    return y_true

In [39]:
hamiltonian_cycles_padded[0].shape[0]

24

In [37]:
embed_dim = 256
latent_dim = 2048
num_heads = 2
batch_size=2


barnette_graph = keras.Input(shape=(barnette_graphs_trn.shape[1],), dtype="int64")
x_1 = PositionalEmbedding(2*max_barnette_len, len(barnette_vocab), embed_dim)(barnette_graph)
x_2 = TransformerEncoder(embed_dim, latent_dim, num_heads)(x_1)
x_3 = keras.layers.Flatten()(x_2)
x_4 = keras.layers.Dense(132, activation='relu')(x_3)
hamiltonian_cycle_logits = keras.layers.Dense(hamiltonian_cycles_padded[0].shape[0], activation='sigmoid')(x_4)
boundary_interior_prop_predictor= keras.layers.Dense(2, activation='sigmoid')(hamiltonian_cycle_logits)

In [38]:
boundary_interior_prop = keras.Model(inputs=barnette_graph, outputs=boundary_interior_prop_predictor)

In [39]:
boundary_interior_prop.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 401)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ positional_embedding                 │ (None, 401, 256)            │         192,000 │
│ (PositionalEmbedding)                │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_encoder                  │ (None, 401, 256)            │       1,577,984 │
│ (TransformerEncoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 102656)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 132)                 │      13,550,724 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 24)                  │           3,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 2)                   │              50 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,323,950 (58.46 MB)

 Trainable params: 15,323,950 (58.46 MB)

 Non-trainable params: 0 (0.00 B)

In [40]:
logits_predictor=keras.Model(inputs=barnette_graph, outputs=hamiltonian_cycle_logits)

In [41]:
logits_predictor.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 401)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ positional_embedding                 │ (None, 401, 256)            │         192,000 │
│ (PositionalEmbedding)                │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_encoder                  │ (None, 401, 256)            │       1,577,984 │
│ (TransformerEncoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 102656)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 132)                 │      13,550,724 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 24)                  │           3,192 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,323,900 (58.46 MB)

 Trainable params: 15,323,900 (58.46 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
batchsize=50
num_epochs=2

opt = tf.keras.optimizers.Adam(learning_rate=0.00001)

for epoch in range(num_epochs):
    for step in tqdm(range(barnette_graphs_trn.shape[0]//batchsize)):
        
        start_idx = batchsize*step
        end_idx = batchsize*(step+1)
        
        X_batch =barnette_graphs_trn[start_idx:end_idx]
        padded =barnette_graphs_padded_trn[start_idx:end_idx]
        #y_true= pseudolabel_trn[start_idx:end_idx]
        
        with tf.GradientTape() as tape:
            y_pred = boundary_interior_prop(X_batch)
            y_true= y_true_calculator(logits_predictor(X_batch),padded)
            
            bce = keras.losses.BinaryCrossentropy(reduction=None)
            loss = bce(y_true, y_pred)
        
        grads = tape.gradient(loss, boundary_interior_prop.trainable_variables)
        opt.apply_gradients(zip(grads, boundary_interior_prop.trainable_variables))
        
    print(str(epoch+1)+" complete with average loss = "+str(float(tf.math.reduce_mean(loss))))

  0%|          | 0/700 [00:00<?, ?it/s]

In [ ]:
#model.save('barnette_to_hamiltonian.keras')

In [ ]:
#from google.colab import files
#files.download('barnette_to_hamiltonian.keras')